In [62]:
import urllib
import urllib.request
import json
import pandas as pd

url = 'https://ussouthcentral.services.azureml.net/workspaces/5e279eff6a844bbe857435d4896e4b78/services/3d48b2c8f66245069a596a602cb7e500/execute?api-version=2.0&details=true'
api_key = 'GgoR7dMdgFe7bNVHGjEMa99QdchTI8yRcZHVwtj3QR4hqBaP2FZN7LyzMYLkLRwJ51CqBbpZaCVJLS41kmJsfA==' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

productIds = pd.read_csv("product_ids.csv")
reviews = pd.read_csv("amazon_reviews.csv")

reviews["sentiment"] = "I have not changed"

# iterate over rows with iterrows()
for index, row in reviews.iterrows():
    data =  {

        "Inputs": {

                "input1":
                {
                    "ColumnNames": ["Col2"],
                    "Values": [ [ row['reviews.text'] ] ]
                },        },
            "GlobalParameters": {
        }
    }
    #Encode our data input into json
    body = str.encode(json.dumps(data))
    #Make API call to provided url
    req = urllib.request.Request(url, body, headers)
    #Open our HTTP object
    response = urllib.request.urlopen(req)
    # Convert HTTP object from bytes to text
    resp_text = urllib.request.urlopen(req).read().decode('UTF-8')
    # Use loads to decode from text
    json_obj = json.loads(resp_text)
    #Parse Object and assign to sentiment column
    reviews.at[index,'sentiment'] = json_obj['Results']['output1']['value']['Values'][0][1]
    #Print sentiment for that review
    print(reviews.at[index, 'sentiment'])
    

0.174385905265808
0.178197503089905
0.277854412794113
0.274751305580139
0.167387634515762
0.178212478756905
0.174946367740631
0.265861511230469
0.153462544083595
0.237073600292206
0.245873749256134
0.204578205943108
0.213741719722748
0.216317504644394
0.200016885995865
0.243389055132866
0.132972344756126


KeyboardInterrupt: 

In [61]:

#Recasts our object as a float datatype
reviews['sentiment'] = reviews['sentiment'].astype(float)
#Aggregates the Sentiment grouped by asin number
sentimentbyAsins = reviews.groupby('asins').agg({'sentiment' : ['max', 'min', 'mean']}).reset_index()
#Renames the columns of our resulting dataframe
sentimentbyAsins.columns = ['asins', 'max', 'min', 'mean']
#Assigns sentiment score to productsId dataset joining on asins
merged_inner = pd.merge(left=productIds,right=sentimentbyAsins, left_on='asins', right_on='asins')
#Compare the two dataframe
print(sentimentbyAsins)
print(merged_inner)


         asins       max       min      mean
0   B00IOY8XWQ  0.519931  0.064239  0.204787
1   B00IOYAM4I  0.300774  0.095642  0.210618
2   B00QFQRELG  0.280988  0.132622  0.205588
3   B00REQKWGA  0.387073  0.065088  0.203724
4   B00VINDBJK  0.387073  0.065088  0.218961
5   B00ZV9PXP2  0.393474  0.065047  0.206710
6   B010CEHQTG  0.685215  0.055817  0.207067
7   B017JG41PC  0.368145  0.065047  0.206710
8   B0189XYY0Q  0.408424  0.080635  0.216130
9   B018Y224PY  0.387067  0.067629  0.217939
10  B018Y225IA  0.367744  0.045523  0.210099
11  B018Y22BI4  0.386132  0.045523  0.210185
12  B018Y22C2Y  0.447338  0.059405  0.200528
13  B018Y23MNM  0.442729  0.072663  0.201956
14  B01ACEKAJY  0.409144  0.073090  0.214620
15  B01AHB9C1E  0.409144  0.101033  0.213172
16  B01AHB9CN2  0.384484  0.047632  0.205150
17  B01AHB9CYG  0.923320  0.060553  0.223070
18  B01AHBBG04  0.367643  0.075767  0.198651
19  B01AHBDCKQ  0.923320  0.073090  0.221868
20  B01BH83OOM  0.450916  0.035833  0.211829
21  B01J24

"\nproductIds['max'] = sentimentbyAsins['max']\nproductIds['min'] = sentimentbyAsins['min']\nproductIds['mean'] = sentimentbyAsins['mean']\nprint(productIds)\n"